# Post-wildfire vegetation regrowth analysis
Author: Stepan Bryleev

### Canopy Height Model 
In this Jupyter Notebook there is a JavaScript code for calculating and displaying Canopy Height Models (CHMs) for the study area in GEE using NEON Lidar data.

```js
// ====================================================== //
// This part of the project is for creating GRSM CHM maps //
// ====================================================== //

// Specify study polygon
var study_polygon = /* color: #98ff00 */ee.Geometry.Polygon(
        [[[-83.5425714556907, 35.701602862979236],
          [-83.5422281329368, 35.690310616971246],
          [-83.53879490539774, 35.63828989061714],
          [-83.5037759844993, 35.615406781430224],
          [-83.48592430887022, 35.61938453875363],
          [-83.47459355041727, 35.627546807436595],
          [-83.47922808124717, 35.65117980654369],
          [-83.47738302453864, 35.682051678012314],
          [-83.4774257678571, 35.69326203209512],
          [-83.47751235961914, 35.71456632200802],
          [-83.47837061486136, 35.7182597023649],
          [-83.48043060302734, 35.72055934657099],
          [-83.53656387329102, 35.70494866628485]]]);

// Define the GRSM perimeter variable
var grsm_boundary = ee.FeatureCollection('projects/ee-stbr4432/assets/grsm_polygon');

// Define the fire perimeter variable
var fire_perimeter = ee.FeatureCollection('projects/ee-stbr4432/assets/chimney_tops_perim');
var fireBoundGeom = fire_perimeter.geometry();

// Apply the intersection method to the Polygon object.
var polygonIntersection = fireBoundGeom.intersection(grsm_boundary);

// Define the study area
var studyArea = study_polygon.intersection(polygonIntersection);

// Set center and zoom 
Map.setCenter(-83.5, 35.6, 11);


// == Create a function to smooth out the CHM-difference raster == //
// ==           (filter out high-frequency patterns)           ==  //

var boxcar = ee.Kernel.square({
  radius: 1.5, units: 'pixels', normalize: true
});


// === Create a function to plot histogram charts === //

// Define a function
function plot_hist(chm_image, studyArea, scale, title) {
  var chm_hist = ui.Chart.image.histogram({image: chm_image, region: studyArea, scale: scale, minBucketWidth: 2})
      .setOptions({title: title,
            hAxis: {title: 'CHM Value (m)',titleTextStyle: {italic: false, bold: true},},
            vAxis: {title: 'Count', titleTextStyle: {italic: false, bold: true}},});
            
  print(chm_hist);
}


// ======= Read data, calculate and plot dCHM ======= //

// Read NEON GRSM CHM data for 2016, 2017, 2021
var neon_canopy_data = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30015-001');
var grsmSiteData = neon_canopy_data.filter('NEON_SITE == "GRSM"');

// Define CHM data for 2016, 2017, 2021
var grsm_chm_2016 = grsmSiteData.filterDate('2016-01-01','2017-01-01').mosaic().clip(studyArea);
var grsm_chm_2017 = grsmSiteData.filterDate('2017-01-01','2018-01-01').mosaic().clip(studyArea);
var grsm_chm_2021 = grsmSiteData.filterDate('2021-01-01','2022-01-01').mosaic().clip(studyArea);

// Calculate CHMs of difference
var dCHM_2017_2016 = grsm_chm_2017.subtract(grsm_chm_2016);
var dCHM_2021_2016 = grsm_chm_2021.subtract(grsm_chm_2016);
var dCHM_2021_2017 = grsm_chm_2021.subtract(grsm_chm_2017);

// Apply the smoothing function to dCHM images by convolving with the boxcar kernel
var dCHM_2017_2016_smooth = dCHM_2017_2016.convolve(boxcar);
var dCHM_2021_2016_smooth = dCHM_2021_2016.convolve(boxcar);
var dCHM_2021_2017_smooth = dCHM_2021_2017.convolve(boxcar);


// Set visualization parameters for CHM and dCHM images
var chm_vis = {min:0, max:40, palette:['lightgreen', 'green']};

// Add CHM layers to the map
Map.addLayer(grsm_chm_2016, chm_vis, 'CHM_2016', 0);
Map.addLayer(grsm_chm_2017, chm_vis, 'CHM_2017', 0);
Map.addLayer(grsm_chm_2021, chm_vis, 'CHM_2021', 0);


// Calculate Mean Canopy Height by year
print('Mean Canopy Height 2016',grsm_chm_2016.reduceRegion({
      reducer: ee.Reducer.mean(),
      geometry: studyArea,
      scale : 250
      }));
      
print('Mean Canopy Height 2017',grsm_chm_2017.reduceRegion({
      reducer: ee.Reducer.mean(),
      geometry: studyArea,
      scale : 250
      }));
      
print('Mean Canopy Height 2021',grsm_chm_2021.reduceRegion({
      reducer: ee.Reducer.mean(),
      geometry: studyArea,
      scale : 250
      }));


// ==== CHM difference classification and visualization ==== //

// Define an SLD style of discrete intervals to apply to the CHM image
var chm_sld_intervals =
  '<RasterSymbolizer>' +
    '<ColorMap type="intervals" extended="false" >' +
      '<ColorMapEntry color="#008000" quantity="100" label="100"/>' +
      '<ColorMapEntry color="#7a8737" quantity="3" label="3"/>' +
      '<ColorMapEntry color="#acbe4d" quantity="2" label="2" />' +
      '<ColorMapEntry color="#c0c0c0" quantity="1" label="1" />' +
      '<ColorMapEntry color="#0ae042" quantity="0" label="0" />' +
      '<ColorMapEntry color="#fff70b" quantity="-1" label="-1" />' +
      '<ColorMapEntry color="#ffaf38" quantity="-2" label="-2" />' +
      '<ColorMapEntry color="#ff641b" quantity="-4" label="-3" />' +
      '<ColorMapEntry color="#9370db" quantity="-6" label="-4" />' +
      '<ColorMapEntry color="#800000" quantity="-10" label="-10" />' +
    '</ColorMap>' +
  '</RasterSymbolizer>';

// Add the image to the map using both the color ramp and interval schemes
Map.addLayer(dCHM_2017_2016_smooth.sldStyle(chm_sld_intervals), {}, 'Smoothed & Classified CHM Diff 2017-2016', 1);
Map.addLayer(dCHM_2021_2016_smooth.sldStyle(chm_sld_intervals), {}, 'Smoothed & Classified CHM Diff 2021-2016', 1);
Map.addLayer(dCHM_2021_2017_smooth.sldStyle(chm_sld_intervals), {}, 'Smoothed & Classified CHM Diff 2021-2017', 1);


// ======== ADD A CHM DIFFERNCE LEGEND ======== //

// Set position of panel
var CHMlegend = ui.Panel({
  style: {
    position: 'bottom-right',
    padding: '8px 15px'
  }});
 
// Create legend title
var CHMlegendTitle = ui.Label({
  value: 'CHM Change',
  style: {fontWeight: 'bold',
    fontSize: '18px',
    margin: '0 0 4px 0',
    padding: '0'
    }});
 
// Add the title to the panel
CHMlegend.add(CHMlegendTitle);
 
// Creates and styles 1 row of the legend.
var makeRow = function(color, name) {
 
      // Create the label that is actually the colored box.
      var colorBox = ui.Label({
        style: {
          backgroundColor: '#' + color,
          // Use padding to give the box height and width.
          padding: '8px',
          margin: '0 0 4px 0'
        }});
 
      // Create the label filled with the description text.
      var description = ui.Label({
        value: name,
        style: {margin: '0 0 4px 6px'}
      });
 
      // Return the panel
      return ui.Panel({
        widgets: [colorBox, description],
        layout: ui.Panel.Layout.Flow('horizontal')
      })};
 
//  Palette with the colors
var palette =['008000', '7a8737', 'acbe4d', 'c0c0c0', '0ae042', 'fff70b', 'ffaf38', 'ff641b', 'a41fd6', '800000'];
 
// Names of the legend
var names = ['> 3 m gain', '2 - 3 m gain', '1 - 2 m gain', '0 - 1 m gain', '0 - 1m loss', '1 - 2 m loss', '2 - 4 m loss',
'4 - 6 m loss', '6 - 10 m loss', '> 10 m loss'];
 
// Add color and names
for (var i = 0; i < 10; i++) {
  CHMlegend.add(makeRow(palette[i], names[i]));
  }  
  
// Add legend to map
Map.add(CHMlegend);


// Display the fire boundary
Map.addLayer(fire_perimeter.style({width: 2,
                                  color: "red",
                                  fillColor: "#00000000"}),{},"Fire Boundary", 1);


// ======= Plot histogram charts ======= //

// Plot histogram charts for each CHM image in the console
plot_hist(grsm_chm_2016, studyArea, 50, 'CHM Histogram, 2016');
plot_hist(grsm_chm_2017, studyArea, 50, 'CHM Histogram, 2017');
plot_hist(grsm_chm_2021, studyArea, 50, 'CHM Histogram, 2021');
```